# Auswertung der Parquet Dateien aus dem Echtzeitarchiv V14

In [1]:
import duckdb
import pandas as pd
import para
#import openpyxl
#from lonboard import Map, HeatmapLayer
import datetime as dt

In [2]:
pd.options.display.max_columns = 100

In [3]:
jetzt = dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
gestern= (dt.date.today() - dt.timedelta(1)).strftime('%Y-%m-%d')
letzte07tage= (dt.date.today() - dt.timedelta(7)).strftime('%Y-%m-%d')
letzte14tage= (dt.date.today() - dt.timedelta(14)).strftime('%Y-%m-%d')
letzte21tage= (dt.date.today() - dt.timedelta(21)).strftime('%Y-%m-%d')

print(jetzt, letzte21tage)

2024-09-22 07:17:39 2024-09-01


# Aufbau der class

In [4]:

class rt_duck:   
    """ERstellen einer Klasse für Echtzeitdaten """ 
    
    def __init__(self, db_name=':memory:'):
        # Initialize the DuckDB connection
        self.conn = duckdb.connect(database=db_name)
        self.cursor = self.conn.cursor()
        self.cursor.sql(f"""INSTALL postgres;
                            LOAD postgres;
                            ATTACH 'dbname=zvbn_postgis user=postgres host=127.0.0.1 password={para.key_dm_db}' AS db_dm (TYPE POSTGRES, READ_ONLY);"""
                        )
        self.cursor.sql("create or replace table lin_buendel as select * from db_dm.basis.lin_buendel")
        sql_lin = """
        Create or replace table linien as 
        SELECT nummer AS linie, buendel, ebene, dlid, id 
        FROM db_dm.basis.linien 
        WHERE buendel IS NOT NULL AND aktiv IS TRUE 
        ORDER BY buendel, ebene, nummer """
        self.cursor.sql(sql_lin)

    #def connect_zvbn_postgis(self):


    def create_table_fahrten(self, server):
        """ erstellt eine Tabelle aus den Parquet Files"""
        sql_create = f"create or replace table fahrten as select * from read_parquet('out/parquet/{server}/fahrten*.parquet',  union_by_name = true, filename = true)"
        self.cursor.execute(sql_create)
        self.cursor.sql("alter table fahrten add column if not exists lineid_short VARCHAR")
        self.cursor.sql("""update fahrten 
                set lineid_short = concat_ws(':', split_part(lineid,':', 1), split_part(lineid,':', 2), split_part(lineid,':', 3))""")
        self.cursor.sql("""select distinct lineid, 
                concat_ws(':', split_part(lineid,':', 1), split_part(lineid,':', 2), split_part(lineid,':', 3)) 
                from fahrten""")

        print("Table 'fahrten' created.")

    def create_vw_buendel(self, buendel):
        """ erstellt sicht auf eine Unternehmen"""
        sql_buendel = f"""create or replace view vw_buendel as
                                (select f.datum, l.buendel, l.ebene, f.vu, f.fnr, f.lineshort,f.lineid_short, f.hasrealtime, 
                                f.journey_cancelled, f.reported_cancelled, f.ts_reported_cancelled
        
                                from fahrten f                                         
                                left outer join linien l on f.lineid_short = l.dlid 
                                 where buendel like '%{buendel}%') """
        self.cursor.execute(sql_buendel)
        

    def anzahl_fahrten(self, table_view):        
        return self.cursor.sql(f"from {table_view}").shape[0]

In [5]:
rt = rt_duck()

In [6]:
result_list = rt.cursor.sql("from lin_buendel")['buendel'].fetchall()
flat_list = [row[0] for row in result_list]
print(flat_list)

['CUX 6', 'CUX 3', 'NachtEule Oldenburg', 'Nachtschwärmer', 'VER Südwest', 'OL West', 'OHZ West', 'AM Ost', 'OHZ Ost', 'NachtEule Ammerland', 'RSB', 'AM Süd', 'AM West', 'BHV', 'DEL', 'DH Nord', 'DH Nordost', 'DH Nordwest', 'DH Süd', 'DH Südost', 'DH Südwest', 'HB Stadt', 'HB Straßenbahn', 'OHZ Mitte', 'OL Nord', 'OL Stadt', 'OL Südost', 'WM Nord', 'WM Süd', 'nahsh', 'REKREUZ', 'RBVBN', 'VER Ost', 'VER Nord']


In [7]:
rt.create_table_fahrten(server = 'prod')

Table 'fahrten' created.


## Erstellen eines Bündels welches die Daten Am West enthält

In [8]:
rt.create_vw_buendel('AM West')

In [9]:
q = rt.cursor.sql("select * from vw_buendel")
q.filter("lineshort = 'S35'")

┌─────────────────────┬─────────┬─────────┬───────────────┬─────────┬───────────┬──────────────┬─────────────┬───────────────────┬────────────────────┬───────────────────────┐
│        datum        │ buendel │  ebene  │      vu       │   fnr   │ lineshort │ lineid_short │ hasRealtime │ journey_cancelled │ reported_cancelled │ ts_reported_cancelled │
│    timestamp_ns     │ varchar │ varchar │    varchar    │ varchar │  varchar  │   varchar    │   boolean   │      boolean      │      boolean       │        varchar        │
├─────────────────────┼─────────┼─────────┼───────────────┼─────────┼───────────┼──────────────┼─────────────┼───────────────────┼────────────────────┼───────────────────────┤
│ 2024-09-14 00:00:00 │ AM West │ 1+      │ Gerdes Reisen │ 6352301 │ S35       │ de:VBN:S35   │ true        │ false             │ false              │                       │
│ 2024-09-14 00:00:00 │ AM West │ 1+      │ Gerdes Reisen │ 6352300 │ S35       │ de:VBN:S35   │ true        │ false    

In [10]:
rt.anzahl_fahrten('vw_buendel')

7733

# Ohne Klassen

In [11]:
con = duckdb.connect()

In [12]:
con.sql(f"""INSTALL postgres;
LOAD postgres;
ATTACH 'dbname=zvbn_postgis user=postgres host=127.0.0.1 password={para.key_dm_db}' AS db_dm (TYPE POSTGRES, READ_ONLY);""")

In [13]:
con.sql("create or replace table lin_buendel as select * from db_dm.basis.lin_buendel")
con.sql("select * from lin_buendel")

┌─────────────────────┬────────────┬───────────────────────┬───────┬───────────────────────────────────────────┬────────────┬────────┬─────────┬─────────────────────┬────────────────────────────────────────────────────────────────┬─────────────┬───────────────────┐
│       buendel       │   harmon   │         kreis         │  id   │                 bemerkung                 │ kreis_kurz │ lfd_nr │  aktiv  │    buendel_lang     │                          project_vms                           │ rt_operator │  betreiber_kurz   │
│       varchar       │    date    │        varchar        │ int32 │                  varchar                  │  varchar   │ int32  │ boolean │       varchar       │                            varchar                             │   varchar   │      varchar      │
├─────────────────────┼────────────┼───────────────────────┼───────┼───────────────────────────────────────────┼────────────┼────────┼─────────┼─────────────────────┼────────────────────────────────────

In [14]:
sql_lin = """
        Create or replace table linien as 
        SELECT nummer AS linie, buendel, ebene, dlid, id 
        FROM db_dm.basis.linien 
        WHERE buendel IS NOT NULL AND aktiv IS TRUE 
        ORDER BY buendel, ebene, nummer """
con.sql(sql_lin)
con.sql("select * from linien")

┌─────────┬─────────┬─────────┬───────────────┬───────┐
│  linie  │ buendel │  ebene  │     dlid      │  id   │
│ varchar │ varchar │ varchar │    varchar    │ int32 │
├─────────┼─────────┼─────────┼───────────────┼───────┤
│ 330     │ AM Ost  │ 1+      │ de:VBN:330    │     7 │
│ 340     │ AM Ost  │ 1+      │ de:VBN:340    │   348 │
│ 370     │ AM Ost  │ 2       │ de:VBN:370    │    59 │
│ 331     │ AM Ost  │ 3       │ de:VBN:331    │   287 │
│ 332     │ AM Ost  │ 3       │ de:VBN:332    │   231 │
│ 333     │ AM Ost  │ 3       │ de:VBN:333    │   232 │
│ 334     │ AM Ost  │ 3       │ de:VBN:334    │   233 │
│ 335     │ AM Ost  │ 3       │ de:VBN:335    │   234 │
│ 336     │ AM Ost  │ 3       │ de:VBN:336    │   235 │
│ 337     │ AM Ost  │ 3       │ de:VBN:337    │   360 │
│  ·      │   ·     │ ·       │     ·         │    ·  │
│  ·      │   ·     │ ·       │     ·         │    ·  │
│  ·      │   ·     │ ·       │     ·         │    ·  │
│ 448     │ WM Süd  │ 3       │ de:VBN:448    │ 

## Abruf der Parquet Files (Tagespakete)

In [15]:
server = 'prod'
con.sql(f"create or replace table fahrten as select * from read_parquet('out/parquet/{server}/fahrten*.parquet',  union_by_name = true, filename = true)")
con.sql(f"create or replace table verlauf as select * from read_parquet('out/parquet/{server}/verlauf*.parquet',  union_by_name = true, filename = true)")
con.sql(f"create or replace table zusatz as select * from read_parquet('out/parquet/{server}/zusatz*.parquet',  union_by_name = true, filename = true)")

### Ermitteln und Löschen von nicht gewollten Betreibern

In [16]:
con.sql("select distinct vu from fahrten where vu like '%Weser%'")

┌─────────────────────────────────────────────────────────────┐
│                             vu                              │
│                           varchar                           │
├─────────────────────────────────────────────────────────────┤
│ Eisenbahnen und Verkehrsbetriebe Elbe-Weser GmbH            │
│ Weser-Ems-Bus Betrieb Bremen                                │
│ Weser-Ems-Bus Auftragnehmerleistungen                       │
│ Eisenbahnen und Verkehrsbetriebe Elbe-Weser GmbH(Bahndaten) │
│ MEW Mobilitätszentrale Elbe-Weser                           │
│ Verkehrsbetriebe Wesermarsch GmbH                           │
└─────────────────────────────────────────────────────────────┘

In [17]:
if False: #True / False um ggf. weiterhin alles durchlaufen zu lassen
    print('Löschen von Betreibern')
    con.sql("delete from fahrten where vu not in ('Weser-Ems-Bus Betrieb Bremen', 'Weser-Ems-Bus Auftragnehmerleistungen')")
    #con.sql("delete from verlauf where vu not in ('Weser-Ems-Bus Betrieb Bremen', 'Weser-Ems-Bus Auftragnehmerleistungen')")
    #con.sql("delete from zusatz where vu not in ('Weser-Ems-Bus Betrieb Bremen', 'Weser-Ems-Bus Auftragnehmerleistungen')")

In [18]:
con.sql(f"select count(*) from fahrten where datum >= '{letzte14tage}'").df().values.tolist()[0][0]

210899

In [19]:
anzahl_fahrten = con.sql(f"select count(*) from fahrten where datum >= '{letzte14tage}'").df().values.tolist()[0][0]
print(f"""Anzahl Fahrten: {anzahl_fahrten},  Länge Verlauf: {con.sql("select count(*) from verlauf").df().values.tolist()[0][0]}    """) 

Anzahl Fahrten: 210899,  Länge Verlauf: 8598421    


In [20]:
con.sql("""select 
            datum, 
            fahrtstartstationname, 
           strftime( cast(fahrtstarttime as TIMESTAMPTZ), '%H:%M') as fahrtstart,
           fahrtendstationname,
           strftime( cast(fahrtendtime as TIMESTAMPTZ), '%H:%M') as fahrtende,
            
            deviceid, 
            split_part(deviceid, '-', 2) as fnr, 
            cast(((cast(split_part(split_part(deviceid, '-', 3), '#', 1) as int64) - 8000000000000) / 1000) as int64) as m2, 
        from fahrten 
        where deviceid like '%680%DBRB%' and datum = '2024-08-29'
        order by datum, fahrtstarttime
        
        """).df()
#.to_excel('out/web.xlsx', index=False)

,datum,fahrtstartstationname,fahrtstart,fahrtendstationname,fahrtende,deviceid,fnr,m2
0,2024-08-29,Wallhöfen Am Mühlenberg,04:08,Bremen Gröpelingen,05:36,0829-1680002-8012306400000#!ADD!#DBRB#,1680002,12306400
1,2024-08-29,Bremen Gröpelingen,05:08,Wallhöfen Am Mühlenberg,06:32,0829-1680001-8012303800000#!ADD!#DBRB#,1680001,12303800
2,2024-08-29,Wallhöfen Am Mühlenberg,05:24,Bremen Gröpelingen,06:52,0829-1680004-8012306401000#!ADD!#DBRB#,1680004,12306401
3,2024-08-29,Vollersode-Giehlermühlen,06:20,Wallhöfen Am Mühlenberg,06:46,0829-1686602-8012316800000#!ADD!#DBRB#,1686602,12316800
4,2024-08-29,Wallhöfen Am Mühlenberg,06:46,Bremen Gröpelingen,08:26,0829-1680006-8012308200000#!ADD!#DBRB#,1680006,12308200
5,2024-08-29,Bremen-Burg(Bus),06:53,Osterholz-Scharmbeck Bahnhof (Bus),07:40,0829-1680005-8012305800000#!ADD!#DBRB#,1680005,12305800
6,2024-08-29,Wallhöfen Am Mühlenberg,07:46,Bremen Gröpelingen,09:21,0829-1680008-8012308300000#!ADD!#DBRB#,1680008,12308300
7,2024-08-29,Osterholz-Scharmbeck Bahnhof (Bus),08:00,Bremen-Burg(Bus),08:39,0829-1680302-8012307400000#!ADD!#DBRB#,1680302,12307400
8,2024-08-29,Bremen Gröpelingen,08:08,Wallhöfen Am Mühlenberg,09:32,0829-1680007-8012303700000#!ADD!#DBRB#,1680007,12303700
9,2024-08-29,Wallhöfen Am Mühlenberg,08:24,Bremen Gröpelingen,09:52,0829-1680010-8012306500000#!ADD!#DBRB#,1680010,12306500


### Anzahl der Fahrten je Betreiber

In [21]:
con.sql("select  journeyOperator, count(journeyOperator) as count from verlauf group by journeyOperator order by count")

┌─────────────────────────────────────────────────────────────┬─────────┐
│                       journeyOperator                       │  count  │
│                           varchar                           │  int64  │
├─────────────────────────────────────────────────────────────┼─────────┤
│ MEW Mobilitätszentrale Elbe-Weser                           │    5764 │
│ Autobus Stoss GmbH                                          │    7951 │
│ SBV Janßen GmbH & Co. KG                                    │   11023 │
│ WestfalenBahn                                               │   14993 │
│ Regionalverkehre Start Deutschland (Niedersachsen-Mitte)    │   16538 │
│ Eisenbahnen und Verkehrsbetriebe Elbe-Weser GmbH(Bahndaten) │   20328 │
│ eurobahn                                                    │   20618 │
│ metronom Eisenbahngesellschaft mbH                          │   24388 │
│ Weser-Ems-Bus Auftragnehmerleistungen                       │   26645 │
│ W. Giese Nachf. Omnibusbetrieb GmbH 

## Fahrten mit hohen Verspätungen

In [22]:
con.sql("select distinct deviceid from verlauf where dep_del > 100").df()

,deviceid
0,0912-82016#!ADD!#me#
1,0916-1283010#!ADD!#IVU-Regio#
2,0912-591004-00004-1#!ADD!#BSAG#
3,0912-6255014#!ADD!#IVU-Regio#
4,0918-4429-800295-8001168-172300#!ADD!#DB#
...,...
189,0905-4402-800295-8000152-052000#!ADD!#DB#
190,0905-3021028-00028-1#!ADD!#BSAG#
191,0911-1260025#!ADD!#IVU-Regio#
192,0915-356007-00007-1#!ADD!#BSAG#


In [23]:
con.sql("describe fahrten")

┌─────────────────────────────┬──────────────┬─────────┬─────────┬─────────┬─────────┐
│         column_name         │ column_type  │  null   │   key   │ default │  extra  │
│           varchar           │   varchar    │ varchar │ varchar │ varchar │ varchar │
├─────────────────────────────┼──────────────┼─────────┼─────────┼─────────┼─────────┤
│ datum                       │ TIMESTAMP_NS │ YES     │ NULL    │ NULL    │ NULL    │
│ fnr                         │ VARCHAR      │ YES     │ NULL    │ NULL    │ NULL    │
│ destination                 │ VARCHAR      │ YES     │ NULL    │ NULL    │ NULL    │
│ hasRealtime                 │ BOOLEAN      │ YES     │ NULL    │ NULL    │ NULL    │
│ vu                          │ VARCHAR      │ YES     │ NULL    │ NULL    │ NULL    │
│ lineid                      │ VARCHAR      │ YES     │ NULL    │ NULL    │ NULL    │
│ lineid_short                │ VARCHAR      │ YES     │ NULL    │ NULL    │ NULL    │
│ lineshort                   │ VARCHAR    

### Verkürzung der DLID
- Zum Teil weren bei mehreren Betreibern einer Linie TLID mit vierteiliger DLID geliefert 
- Verkürzung ermöglicht die Verknüpfung mit Liste aus DM

In [24]:
con.sql("alter table fahrten add column if not exists lineid_short VARCHAR")
con.sql("""update fahrten 
        set lineid_short = concat_ws(':', split_part(lineid,':', 1), split_part(lineid,':', 2), split_part(lineid,':', 3))""")
con.sql("""select distinct lineid, 
        concat_ws(':', split_part(lineid,':', 1), split_part(lineid,':', 2), split_part(lineid,':', 3)) 
        from fahrten""")

┌───────────────┬────────────────────────────────────────────────────────────────────────────────────────────────────┐
│    lineid     │ concat_ws(':', split_part(lineid, ':', 1), split_part(lineid, ':', 2), split_part(lineid, ':', 3)) │
│    varchar    │                                              varchar                                               │
├───────────────┼────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ de:VBN:95:    │ de:VBN:95                                                                                          │
│ de:VBN:20:1   │ de:VBN:20                                                                                          │
│ de:VBN:S:VGB  │ de:VBN:S                                                                                           │
│ de:VBN:4S:    │ de:VBN:4S                                                                                          │
│ de:VBN:AST41: │ de:VBN:AST41                  

## Über HIM gemeldete Ausfälle (ts_reported_cancelled gefüllt)

In [25]:
df_fahrten_ausfall_him = con.sql(f"""
                              select vu, fnr, ts_reported_cancelled, journey_cancelled 
                              from fahrten f 
                              where ts_reported_cancelled != '' and f.datum >= '{letzte14tage}'""").df()

## Echzeitquote

### nach Linie und Betreiber

In [26]:
df_ez_quote_betreiber = con.sql(f"""
        select l.buendel, l.ebene,f.datum, f.vu, f.lineshort,f.lineid_short, count(f.hasRealtime) filter (f.hasRealtime = True) ez_true, count(f.*) count, 
        round(ez_true / count * 100, 1) anteil_ez
        from fahrten f
        left outer join linien l on f.lineid_short = l.dlid
        where f.datum >= '{letzte14tage}'              
        group by f.lineid_short, f.vu, f.datum, f.lineshort, f.lineid_short, l.buendel, l.ebene
        order by f.vu, f.lineid_short
        """).df()
df_ez_quote_betreiber['buendel'] = df_ez_quote_betreiber['buendel'].fillna('-')
df_ez_quote_betreiber['ebene'] = df_ez_quote_betreiber['ebene'].fillna('-')
anteil_ez_pivot_betreiber = pd.pivot_table(df_ez_quote_betreiber, index=['buendel','ebene', 'vu', 'lineshort'], columns='datum', values='anteil_ez').reset_index()
anteil_ez_pivot_betreiber

datum,buendel,ebene,vu,lineshort,2024-09-08 00:00:00,2024-09-09 00:00:00,2024-09-10 00:00:00,2024-09-11 00:00:00,2024-09-12 00:00:00,2024-09-13 00:00:00,2024-09-14 00:00:00,2024-09-15 00:00:00,2024-09-16 00:00:00,2024-09-17 00:00:00,2024-09-18 00:00:00,2024-09-19 00:00:00,2024-09-20 00:00:00,2024-09-21 00:00:00
0,-,-,Autobus Stoss GmbH,558,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,NaN
1,-,-,Autobus Stoss GmbH,559,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,NaN
2,-,-,BREMERHAVEN BUS,518,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-,-,BREMERHAVEN BUS,519,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,NaN
4,-,-,BREMERHAVEN BUS,E01,NaN,100.0,100.0,100.0,100.0,100.0,NaN,NaN,50.0,100.0,100.0,100.0,100.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
685,WM Süd,3,Gebken Reisen GmbH,458,NaN,96.0,100.0,96.0,100.0,90.9,NaN,NaN,68.0,88.0,96.0,88.0,86.4,NaN
686,WM Süd,3,Gebken Reisen GmbH,459,NaN,70.6,76.5,76.5,76.5,50.0,NaN,NaN,55.6,66.7,55.6,61.1,40.0,NaN
687,WM Süd,3,Gebken Reisen GmbH,461,NaN,90.0,90.0,80.0,80.0,88.9,NaN,NaN,50.0,90.0,90.0,90.0,88.9,NaN
688,WM Süd,3,Gebken Reisen GmbH,462,NaN,100.0,100.0,100.0,100.0,100.0,NaN,NaN,66.7,66.7,66.7,100.0,100.0,NaN


### nach Linie (ohne Betreiber)

In [27]:
df_ez_quote_o_betreiber = con.sql(f"""
        select l.buendel, l.ebene,f.datum, f.lineshort,f.lineid_short, count(f.hasRealtime) filter (f.hasRealtime = True) ez_true, count(f.*) count, 
        round(ez_true / count * 100, 1) anteil_ez
        from fahrten f        
        left outer join linien l on f.lineid_short = l.dlid      
        where f.datum >= '{letzte14tage}'        
        group by f.lineid_short, f.datum, f.lineshort, f.lineid_short, l.buendel, l.ebene
        order by f.lineid_short
        """).df()
df_ez_quote_o_betreiber['buendel'] = df_ez_quote_o_betreiber['buendel'].fillna('-')
df_ez_quote_o_betreiber['ebene'] = df_ez_quote_o_betreiber['ebene'].fillna('-')
anteil_ez_pivot_o_betreiber = pd.pivot_table(df_ez_quote_o_betreiber, index=['buendel','ebene', 'lineshort'], columns='datum', values='anteil_ez').reset_index()
anteil_ez_pivot_o_betreiber

datum,buendel,ebene,lineshort,2024-09-08 00:00:00,2024-09-09 00:00:00,2024-09-10 00:00:00,2024-09-11 00:00:00,2024-09-12 00:00:00,2024-09-13 00:00:00,2024-09-14 00:00:00,2024-09-15 00:00:00,2024-09-16 00:00:00,2024-09-17 00:00:00,2024-09-18 00:00:00,2024-09-19 00:00:00,2024-09-20 00:00:00,2024-09-21 00:00:00
0,-,-,1020,NaN,100.0,100.0,100.0,100.0,100.0,0.0,NaN,88.9,100.0,100.0,100.0,100.0,100.0
1,-,-,1021,100.0,96.2,92.3,88.5,88.5,96.2,0.0,0.0,30.8,96.2,96.2,96.2,96.2,66.7
2,-,-,108,NaN,100.0,100.0,100.0,100.0,100.0,NaN,NaN,78.8,100.0,100.0,100.0,100.0,NaN
3,-,-,10E,100.0,100.0,100.0,66.7,100.0,100.0,NaN,NaN,33.3,100.0,100.0,100.0,100.0,NaN
4,-,-,1E,100.0,98.1,98.1,92.5,98.1,100.0,100.0,100.0,73.1,100.0,100.0,100.0,100.0,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
641,WM Süd,3,458,NaN,96.0,100.0,96.0,100.0,90.9,NaN,NaN,68.0,88.0,96.0,88.0,86.4,NaN
642,WM Süd,3,459,NaN,70.6,76.5,76.5,76.5,50.0,NaN,NaN,55.6,66.7,55.6,61.1,40.0,NaN
643,WM Süd,3,461,NaN,90.0,90.0,80.0,80.0,88.9,NaN,NaN,50.0,90.0,90.0,90.0,88.9,NaN
644,WM Süd,3,462,NaN,100.0,100.0,100.0,100.0,100.0,NaN,NaN,66.7,66.7,66.7,100.0,100.0,NaN


## Fahrten ohne Echtzeit Ebene 1/1+ und 2

In [28]:
df_fahrten_ohne_ez_ebenen_1_1p_2 = con.sql(f"""
        select f.datum, l.buendel, l.ebene, f.vu, f.fnr, f.lineshort,f.lineid_short, f.hasrealtime, f.journey_cancelled, f.reported_cancelled, f.ts_reported_cancelled
        
        from fahrten f
                                           
        left outer join linien l on f.lineid_short = l.dlid              
        where l.ebene in ('1', '1+') and f.hasrealtime = False and f.datum >= '{letzte14tage}'
                                           
        order by f.datum, f.lineid_short
        """).df()

In [29]:
df_fahrten_ausfall_1_1p_2 = con.sql(f"""
        select f.datum, l.buendel, l.ebene, f.vu, f.fnr, f.lineshort,f.lineid_short, f.hasrealtime, f.journey_cancelled, f.reported_cancelled, f.ts_reported_cancelled
        
        from fahrten f
                                    
        left outer join linien l on f.lineid_short = l.dlid              
        where l.ebene in ('1', '1+', '2') and (journey_cancelled = True or f.reported_cancelled = True) and 
        f.datum >= '{letzte14tage}'                            
        order by f.datum, f.lineid_short
        """).df()

# Ausgabe Anteil EZ nach Linie

In [30]:
xlsx = "/var/www/rt_archiv/anteil_echtzeit_linien_vbn.xlsx"
sn00 = '00 Hilfe'
sn01 = '01 pivot alle Linien betreiber'
sn02 = '02 pivot alle Linien'
sn03 = '03 fahrten ohne EZ 1 1+ 2'
sn04 = '04 fahrten ohne EZ 1 1+ 3 grup'
sn06 = '05 fahrten ausfall'
sn07 = '06 fahrten ausfall über HIM'
with pd.ExcelWriter(xlsx, engine="openpyxl") as writer:
    #Hilfeblatt
    writer.book.create_sheet(sn00)
    sheet = writer.book[sn00]
    sheet['A1'] = f"Erstellt: {dt.datetime.now().strftime('%Y-%m-%d %H:%M')} Zeitraum: {letzte14tage} bis {gestern}"

    sheet['A3'] = "Inhalt"
    sheet['B4'] = f"Blatt {sn01}: Pivot Echtzeitquote inkl. Betreiberkennung"
    sheet['B5'] = f"Blatt {sn02}: Pivot Echtzeitquote ohne Betreiberkennung"
    sheet['B6'] = f"Blatt {sn03}: Fahrten ohne Echtzeit"
    sheet['B7'] = f"Blatt {sn04}: Fahrten ohne Echtzeit mit Anzahl"
    sheet['B8'] = f"Blatt {sn06}: Fahrten Ausfall"
    sheet['B9'] = f"Blatt {sn07}: Fahrten Ausfall über HIM"

    #mit Kennung der Betreiber
    anteil_ez_pivot_betreiber.to_excel(writer, sheet_name=sn01, index=False)
    writer.book[sn01].freeze_panes = 'e2'
    writer.book[sn01].auto_filter.ref='A:H'
    for cell in writer.book[sn01]["1:1"]:
        cell.number_format = 'YYYY-MM-DD'
    writer.book[sn01].column_dimensions['c'].width = 22
    for c in ['D', 'E', 'F', 'G', 'H']:
        writer.book[sn01].column_dimensions[c].width = 22        
    for c in writer.book[sn01].iter_cols(min_col=4, max_col=anteil_ez_pivot_betreiber.shape[1]+4):
                #ermitteln der Spalte column letter
                cl = c[int(f"{anteil_ez_pivot_betreiber.shape[0]}")].column_letter
                writer.book[sn01].column_dimensions[cl].width = 16

    #Anteil EZ ohne Kennung der Betreiber
    anteil_ez_pivot_o_betreiber.to_excel(writer, sheet_name=sn02, index=False)
    writer.book[sn02].freeze_panes = 'd2'
    writer.book[sn02].auto_filter.ref='A:H'
    for cell in writer.book[sn02]["1:1"]:
        cell.number_format = 'YYYY-MM-DD'
    writer.book[sn02].column_dimensions['c'].width = 22
    for c in ['D', 'E', 'F', 'G', 'H']:
        writer.book[sn02].column_dimensions[c].width = 22 
         
    for c in writer.book[sn02].iter_cols(min_col=4, max_col=anteil_ez_pivot_o_betreiber.shape[1]+4):
                #ermitteln der Spalte column letter
                cl = c[int(f"{anteil_ez_pivot_o_betreiber.shape[0]}")].column_letter
                writer.book[sn02].column_dimensions[cl].width = 16

    ## Ausgabe der Fahrten ohne Echtzeit Ebene 1 und 1+ und 2 einzeln
    df_fahrten_ohne_ez_ebenen_1_1p_2.to_excel(writer, sheet_name=sn03, index=False)
    writer.book[sn03].freeze_panes = 'a2'
    writer.book[sn03].auto_filter.ref='A:M'
    for cell in writer.book[sn03]["A"]:
        cell.number_format = 'YYYY-MM-DD'
    writer.book[sn03].column_dimensions['A'].width = 18

    ## Ausgabe der Fahrten ohne Echtzeit Ebene 1 und 1+ und 2 gruppiert mit Anzahl
    df_fahrten_ohne_ez_ebenen_1_1p_2[['vu', 'fnr']].value_counts().reset_index().sort_values(['count', 'vu'], ascending=False).to_excel(writer, sheet_name=sn04, index=False)
    writer.book[sn04].freeze_panes = 'a2'
    writer.book[sn04].auto_filter.ref='A:H'
    writer.book[sn04].column_dimensions['A'].width = 22   

    ## Ausgabe der Fahrten Ausfall Ebene 1, 1+ und 2
    df_fahrten_ausfall_1_1p_2.to_excel(writer, sheet_name=sn06, index=False)
    writer.book[sn06].freeze_panes = 'a2'
    writer.book[sn06].auto_filter.ref='A:M'
    for cell in writer.book[sn06]["A"]:
        cell.number_format = 'YYYY-MM-DD'
    writer.book[sn06].column_dimensions['A'].width = 18

    ## Ausgabe der Fahrten Ausfall über HIM
    df_fahrten_ausfall_him.to_excel(writer, sheet_name=sn07, index=False)
    writer.book[sn07].freeze_panes = 'a2'
    writer.book[sn07].auto_filter.ref='A:M'
    for cell in writer.book[sn07]["A"]:
        cell.number_format = 'YYYY-MM-DD'
    writer.book[sn07].column_dimensions['A'].width = 18